# Magnetized Spheres

This code calculates some magnetical problems with a spherical simetry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from fatiando import mesher, gridder, utils
from fatiando.gravmag import sphere
from fatiando.mesher import Sphere
from fatiando.vis import mpl
from fatiando.constants import CM, T2NT
from matplotlib import colors, ticker, cm
import mag_sphere_functions as mfun
% matplotlib inline

## Spheres

In [ ]:
inc, dec = -60., 50. # inclination and declination of regional field
incs, decs = 70., -50.      # wrong direction or remanent

# creating the list of spheres
props1 = {'magnetization':utils.ang2vec(10., inc, dec)} # defining induced magnetization

model = [
    [17000, 22000, 3000, 1200,props1],
    [13000, 18000, 3000, 1200,props1]] # list of sphere models [x0,y0,z0,R,props]

## Observation Points

In [ ]:
#area over which the data are calculated
#x minimum, x maximum, y minimum and y maximum
area = [0, 30000, 5000, 35000] 

#number of data along the y and x directions
shape = (80,80)

#total number of data
N = shape[0]*shape[1]

#coordinates x and y of the data
x = np.linspace(area[0],area[1],shape[0]) # points in x
y = np.linspace(area[2],area[3],shape[0]) # points in y
xp,yp = np.meshgrid(x,y)    # creating mesh points
xp = xp.ravel()
yp = yp.ravel()

#vertical coordinates of the data
zp = -350. - 500.*utils.gaussian2d(xp, yp, 17000, 21000, 21000, 18500, angle=21)

In [ ]:
mpl.figure(figsize=(10,10))
mpl.title("Relief (m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, zp, shape, 30, cmap=mpl.cm.Greens_r)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=1.0).set_label('nT')
mpl.m2km()
mpl.show()

### Result for magnetic induction of my function

In [ ]:
Bx = mfun.bx_sphere(xp, yp, zp, model)
By = mfun.by_sphere(xp, yp, zp, model)
Bz = mfun.bz_sphere(xp, yp, zp, model)

### Result for total field anomaly of my function

In [ ]:
tf = mfun.tf_sphere(xp, yp, zp, model, inc, dec)

### Ploting results

In [ ]:
mpl.figure(figsize=(15,15))
mpl.axis('scaled')

mpl.subplot(2,2,1)
mpl.axis('scaled')
mpl.title("TFA (nT)")
mpl.contourf(yp, xp, tf,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(2,2,2)
mpl.axis('scaled')
mpl.title("Bx (nT)")
mpl.contourf(yp, xp, Bx,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(2,2,3)
mpl.axis('scaled')
mpl.title("By (nT)")
mpl.contourf(yp, xp, By,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(2,2,4)
mpl.axis('scaled')
mpl.title("Bz (nT)")
mpl.contourf(yp, xp, Bz,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()


mpl.show()

### Parameters for layer

In [ ]:
R = (3./(4.*np.pi))**(1./3)
h = np.zeros(N) - 100.                   # layer height


### Result for foward problem

In [ ]:
A = mfun.sm_tf_sphere(xp, yp, zp, xp, yp, h ,inc,dec,inc,dec)

In [ ]:
I = np.identity(xp.size)

In [ ]:
f0 = np.trace(np.dot(A.T, A))/xp.size

In [ ]:
u0 = 0.001

In [ ]:
vp = np.linalg.solve(np.dot(A.T,A) + u0*f0*I,np.dot(A.T,tf))

In [ ]:
#model_layer = []
#for xm, ym, hm, Rm, magm in zip(xp,yp,h,np.zeros(N)+R,vp):
#    model_layer.append([xm, ym, hm, Rm, {'magnetization': utils.ang2vec(magm,inc,dec)}])

In [ ]:
tf_layer_fd = np.dot(A,vp)

### Comparison with TFA function

In [ ]:
np.allclose(tf,tf_layer_fd)

In [ ]:
r = tf-tf_layer_fd

np.mean(r), np.std(r)

In [ ]:
mpl.figure(figsize=(15,10))
mpl.subplot(1,3,1)
mpl.title("TFA (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tf,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.m2km()
mpl.subplot(1,3,2)
mpl.title("TFA_fd (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tf_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.subplot(1,3,3)
mpl.title("vp (A/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, vp,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('A/m')
mpl.show()

In [ ]:
A_bx = mfun.sm_bx_sphere(xp, yp, zp, xp, yp, h, incs, decs)

In [ ]:
Bx_layer_fd = np.dot(A_bx,vp)

In [ ]:
np.allclose(Bx,Bx_layer_fd)

In [ ]:
A_by = mfun.sm_by_sphere(xp, yp, zp, xp, yp, h, incs, decs)

In [ ]:
By_layer_fd = np.dot(A_by,vp)

In [ ]:
np.allclose(By,By_layer_fd)

In [ ]:
A_bz = mfun.sm_bz_sphere(xp, yp, zp, xp, yp, h, incs, decs)

In [ ]:
Bz_layer_fd = np.dot(A_bz,vp)

In [ ]:
np.allclose(Bz,Bz_layer_fd)

In [ ]:
#H = np.dot(np.vstack([A, A_bx, A_by, A_bz]).T, np.vstack([A, A_bx, A_by, A_bz]))
#h = np.dot(np.vstack([A, A_bx, A_by, A_bz]).T, np.hstack([tf, Bx, By, Bz]))

In [ ]:
#np.hstack([tf, Bx, By, Bz]).shape

In [ ]:
#vp_joint = np.linalg.solve(H,h)

In [ ]:
#Bx_layer_fd_joint = np.dot(A_bx, vp_joint)
#By_layer_fd_joint = np.dot(A_by, vp_joint)
#Bz_layer_fd_joint = np.dot(A_bz, vp_joint)
#tf_layer_fd_joint = np.dot(A, vp_joint)

In [ ]:
mpl.figure(figsize=(15,15))
mpl.axis('scaled')

mpl.subplot(3,2,1)
mpl.axis('scaled')
mpl.title("Bx - regional direction (nT)")
mpl.contourf(yp, xp, Bx,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(3,2,2)
mpl.axis('scaled')
mpl.title("Bx - wrong direction (nT)")
mpl.contourf(yp, xp, Bx_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(3,2,3)
mpl.axis('scaled')
mpl.title("By - regional direction (nT)")
mpl.contourf(yp, xp, By,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(3,2,4)
mpl.axis('scaled')
mpl.title("By - wrong direction (nT)")
mpl.contourf(yp, xp, By_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(3,2,5)
mpl.axis('scaled')
mpl.title("Bz - regional direction (nT)")
mpl.contourf(yp, xp, Bz,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.subplot(3,2,6)
mpl.axis('scaled')
mpl.title("Bz - wrong direction (nT)")
mpl.contourf(yp, xp, Bz_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar()
mpl.m2km()

mpl.show()

In [ ]:
A_btb = mfun.sm_btb_sphere(xp, yp, zp, xp, yp, h, inc, dec)

In [ ]:
btb = np.dot(A_btb,vp)

In [ ]:
BTB_true = np.sqrt(Bx**2 + By**2 + Bz**2)

In [ ]:
BTB_layer_fd = np.sqrt(Bx_layer_fd**2 + By_layer_fd**2 + Bz_layer_fd**2)

In [ ]:
#A_btbw = mfun.sm_btb_sphere(xp, yp, zp, xp, yp, h, incs, decs)

In [ ]:
#btbw = np.dot(A_btbw,vp)

In [ ]:
mpl.figure(figsize=(15,15))

mpl.subplot(1,3,1)
mpl.title("BTB true (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, BTB_true,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.m2km()

mpl.subplot(1,3,2)
mpl.title("BTB layer (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, BTB_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')

mpl.subplot(1,3,3)
mpl.title("BTB layer fd (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, btb,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')

mpl.show()